In [35]:
!pip install asyncio aiohttp accuweather nest_asyncio hdfs

     |████████████████████████████████| 41 kB 30 kB/s  eta 0:00:01
  Created wheel for hdfs: filename=hdfs-2.5.8-py3-none-any.whl size=33213 sha256=84b99b51932c51a3b9b5757e0c8b6ca8c40ac3432bb1c78227c2e186057bb1ed
  Stored in directory: /home/jovyan/.cache/pip/wheels/43/cb/59/3fdce328ada746ea437798538a9808e4f730135f5a26f137a4
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=f0fa2c136894e6db90b410507a0bc0d9ac2042eb0b71de91a56232e9eff673a8
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built hdfs docopt


In [48]:
import logging 
import asyncio
import nest_asyncio
import time
nest_asyncio.apply()

In [52]:
from accuweather import (
     AccuWeather,
     ApiError,
     InvalidApiKeyError,
     InvalidCoordinatesError,
     RequestsExceededError,
     )

from aiohttp import ClientError, ClientSession
import pandas as pd

In [53]:
url="https://raw.githubusercontent.com/orozcohsu/iii_20201128/main/Starbucks%20Store.csv"
df=pd.read_csv(url)

In [54]:
#註冊api: https://developer.accuweather.com/
#於myapp 建立一個新的app，並取得api_key
API_KEY = "YZkmPCPnwCH1V0L3GXJgF9b5fQChmQJD"

In [55]:
#建立一個非同步的函數
#學習: https://www.maxlist.xyz/2020/03/29/python-coroutine/

async def main(LATITUDE, LONGITUDE):
    async with ClientSession() as websession:
        try:
            accuweather = AccuWeather(
                API_KEY, websession, latitude=LATITUDE, longitude=LONGITUDE
            )
            current_conditions = await accuweather.async_get_current_conditions()
        except (
            ApiError,
            InvalidApiKeyError,
            InvalidCoordinatesError,
            ClientError,
            RequestsExceededError,
        ) as error:
            print("Error: {error}")
        else:

            loc=f"{accuweather.location_name}"
            tem=f"{current_conditions['RealFeelTemperatureShade']['Metric']['Value']}"
            hum=f"{current_conditions['RelativeHumidity']}"
            
            print(f"Location: {accuweather.location_name}")
            print(f"Current Temperature: {current_conditions['RealFeelTemperatureShade']['Metric']['Value']}")
            print(f"Current RelativeHumidity: {current_conditions['RelativeHumidity']}")
        return loc, tem, hum

In [56]:
df_api=pd.DataFrame()

loc_lst=[]
tem_lst=[]
hum_lst=[]
Longitude_lst=[]
Latitude_lst=[]

#取出每個店點的經緯度
for index, row in df[['Longitude','Latitude']].loc[0:3].iterrows():
    loop = asyncio.get_event_loop()
    loc, tem, hum = loop.run_until_complete(main(row['Latitude'], row['Longitude']))
    Longitude_lst.append(row['Longitude'])
    Latitude_lst.append(row['Latitude'])
    loc_lst.append(loc)
    tem_lst.append(tem)
    hum_lst.append(hum)
    time.sleep(3)

#寫入dataframe
df_api['Longitude']=Longitude_lst
df_api['Latitude']=Latitude_lst
df_api['loc']=loc_lst
df_api['tem']=tem_lst
df_api['hum']=hum_lst

Location: Andorra la Vella
Current Temperature: 5.2
Current RelativeHumidity: 65
Location: Ajman
Current Temperature: 25.6
Current RelativeHumidity: 47
Location: Ajman
Current Temperature: 25.6
Current RelativeHumidity: 47
Location: Abu Dhabi
Current Temperature: 27.7
Current RelativeHumidity: 32


In [57]:
#merge dataframe

result=pd.merge(df, df_api, how='left', on=['Longitude', 'Latitude'])
result.head(10)


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,loc,tem,hum
0,Starbucks,47370-257954,"Meritxell, 96",LicensedLicensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,Andorra la Vella,5.2,65
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42,Ajman,25.6,47
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39,Ajman,25.6,47
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,Abu Dhabi,27.7,32
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,NaN,NaN,NaN
5,Starbucks,17688-182164,"Dalma Mall, Ground Floor",Licensed,"Dalma Mall, Mussafah",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.49,24.40,NaN,NaN,NaN
6,Starbucks,18182-182165,"Dalma Mall, Level 1",Licensed,"Dalma Mall, Mussafah",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.49,24.40,NaN,NaN,NaN
7,Starbucks,23359-229184,Debenhams Yas Mall,Licensed,Yas Island,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.61,24.46,NaN,NaN,NaN
8,Starbucks,30781-99022,Khalidiya Street,Licensed,Khalidiya St.,Abu Dhabi,AZ,AE,NaN,26670052,GMT+04:00 Asia/Muscat,55.69,24.19,NaN,NaN,NaN
9,Starbucks,20423-205465,Eastern Mangroves,Licensed,"Al Salam Street, The Mangroves",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,Abu Dhabi,27.7,32


In [84]:
from hdfs import *
import os

client = Client('http://master:50070')

#先刪除目錄，然後再建立目錄
client.delete('/iii-bigdata/Starbucks_Store', recursive=True)
client.makedirs('/iii-bigdata/Starbucks_Store', permission = 777 )

In [85]:
#寫入hdfs
#該文字內有逗號，特別處理將sep改為|，以免混淆
with client.write('/iii-bigdata/Starbucks_Store/Starbucks_Store.csv', encoding='utf-8') as writer:
    result.to_csv(writer, index=0, sep='|', na_rep='NULL')

In [80]:
df

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",LicensedLicensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,21401-212072,Rex,Licensed,"141 Nguyá»…n Huá»‡, Quáº­n 1, GÃ³c Ä‘Æ°á»ng P...",ThÃ nh Phá»‘ Há»“ ChÃ­ Minh,SG,VN,70000,08 3824 4668,GMT+000000 Asia/Saigon,106.70,10.78
25596,Starbucks,24010-226985,Panorama,Licensed,"SN-44, TÃ²a NhÃ Panorama, 208 Tráº§n VÄƒn TrÃ...",ThÃ nh Phá»‘ Há»“ ChÃ­ Minh,SG,VN,70000,08 5413 8292,GMT+000000 Asia/Saigon,106.71,10.72
25597,Starbucks,47608-253804,Rosebank Mall,Licensed,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,27873500159,GMT+000000 Africa/Johannesburg,28.04,-26.15
25598,Starbucks,47640-253809,Menlyn Maine,Licensed,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,NaN,GMT+000000 Africa/Johannesburg,28.28,-25.79


In [65]:
result

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,loc,tem,hum
0,Starbucks,47370-257954,"Meritxell, 96",LicensedLicensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,Andorra la Vella,5.2,65
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42,Ajman,25.6,47
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39,Ajman,25.6,47
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,Abu Dhabi,27.7,32
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,21401-212072,Rex,Licensed,"141 Nguyá»…n Huá»‡, Quáº­n 1, GÃ³c Ä‘Æ°á»ng P...",ThÃ nh Phá»‘ Há»“ ChÃ­ Minh,SG,VN,70000,08 3824 4668,GMT+000000 Asia/Saigon,106.70,10.78,NaN,NaN,NaN
25596,Starbucks,24010-226985,Panorama,Licensed,"SN-44, TÃ²a NhÃ Panorama, 208 Tráº§n VÄƒn TrÃ...",ThÃ nh Phá»‘ Há»“ ChÃ­ Minh,SG,VN,70000,08 5413 8292,GMT+000000 Asia/Saigon,106.71,10.72,NaN,NaN,NaN
25597,Starbucks,47608-253804,Rosebank Mall,Licensed,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,27873500159,GMT+000000 Africa/Johannesburg,28.04,-26.15,NaN,NaN,NaN
25598,Starbucks,47640-253809,Menlyn Maine,Licensed,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,NaN,GMT+000000 Africa/Johannesburg,28.28,-25.79,NaN,NaN,NaN
